# **IgnoreGAN**

In [1]:
!pip install torchbearer

In [0]:
import numpy as np
import datetime
import os, sys, time
import torchbearer
%matplotlib inline
import matplotlib.pyplot as plt
from torchvision.transforms import ToTensor
from skimage import color, data, restoration
from scipy.signal import convolve2d
from PIL import Image
import glob
from scipy.ndimage.filters import gaussian_filter
import torchvision.transforms as transforms
from matplotlib.pyplot import imshow
import itertools
import pickle
import imageio
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable
import torchvision
import random 
import torch.utils.data as data
from astropy.convolution import Gaussian2DKernel
from torch.utils.data import ConcatDataset
from torchvision.utils import save_image
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
from torch.utils.data import Dataset, DataLoader


In [3]:
# Directory for saving the images
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


# **Import data**

In [4]:
# The dataset is created
transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize((0.5,), (0.5,))]
)

mnist_data = datasets.MNIST(root='data', train=True,download = True, transform=transform)

0it [00:00, ?it/s]

9920512it [00:00, 19934291.09it/s]                            


Extracting data/MNIST/raw/train-images-idx3-ubyte.gz


32768it [00:00, 331621.70it/s]
0it [00:00, ?it/s]

Extracting data/MNIST/raw/train-labels-idx1-ubyte.gz


1654784it [00:00, 5186759.30it/s]                           
8192it [00:00, 132259.16it/s]


Extracting data/MNIST/raw/t10k-images-idx3-ubyte.gz
Extracting data/MNIST/raw/t10k-labels-idx1-ubyte.gz
Processing...
Done!


In [0]:
# Create the dataloader 
full_loader = data.DataLoader(mnist_data, batch_size=128,
                                    num_workers = 4,
                                    pin_memory= True )

In [0]:
samples_dir = '/content/gdrive/My Drive/p_0/'

In [0]:
if not os.path.exists(samples_dir):
        os.mkdir(samples_dir)

# **Construct the Generator code**

In [0]:
class ModelG(nn.Module):
    def __init__(self):
        self.z_dim = 100
        super(ModelG, self).__init__()
        self.fc2 = nn.Linear(10, 1000)
        self.fc = nn.Linear(self.z_dim+1000, 64*28*28)
        self.bn1 = nn.BatchNorm2d(64)
        self.deconv1 = nn.ConvTranspose2d(64, 32, 5, 1, 2)
        self.bn2 = nn.BatchNorm2d(32)
        self.deconv2 = nn.ConvTranspose2d(32, 1, 5, 1, 2)
    
    def weight_init(self, mean, std):
        for m in self._modules:
            normal_init(self._modules[m], mean, std)
    def forward(self, x, labels):
        batch_size = x.size(0)
        y_ = self.fc2(labels)
        y_ = F.relu(y_)
        x = torch.cat([x, y_], 1)
        x = self.fc(x)
        x = x.view(batch_size, 64, 28, 28)
        x = self.bn1(x) 
        x = F.relu(x)
        x = self.deconv1(x)
        x = self.bn2(x)
        x = F.relu(x)
        x = self.deconv2(x)
        x = torch.tanh(x)
        return x

# **Construct the Discriminator code**

In [0]:
class ModelD(nn.Module):
    def __init__(self):
        super(ModelD, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 5, 1, 2)
        self.bn1 = nn.BatchNorm2d(32)
        self.conv2 = nn.Conv2d(32, 64, 5, 1, 2)
        self.bn2 = nn.BatchNorm2d(64)
        self.fc1  = nn.Linear(64*28*28+1000, 1024)
        self.fc2 = nn.Linear(1024, 1)
        self.fc3 = nn.Linear(10, 1000)
    
    def weight_init(self, mean, std):
        for m in self._modules:
            normal_init(self._modules[m], mean, std)
           
    def forward(self, x, labels):
        batch_size = x.size(0)
        x = x.view(batch_size, 1, 28,28)
        x = self.conv1(x)
        x = self.bn1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = self.bn2(x)
        x = F.relu(x)
        x = x.view(batch_size, 64*28*28)
        y_ = self.fc3(labels)
        y_ = F.relu(y_)
        x = torch.cat([x, y_], 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        return torch.sigmoid(x)

# **Constructing BetterCNN model for inception score**

In [0]:
# Model Definition
class BetterCNN(nn.Module):
    def __init__(self):
        super(BetterCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 30, (5, 5), padding=0)
        self.conv2 = nn.Conv2d(30, 15, (3, 3), padding=0)
        self.fc1 = nn.Linear(15 * 5**2, 128)
        self.fc2 = nn.Linear(128, 50)
        self.fc3 = nn.Linear(50, 10)
    
    def forward(self, x):
        out = self.conv1(x)
        out = F.relu(out)
        out = F.max_pool2d(out, (2,2))
        out = self.conv2(out)
        out = F.relu(out)
        out = F.max_pool2d(out, (2,2))
        out = F.dropout(out,0.2)
        out = out.view(out.shape[0], -1)
        out = self.fc1(out)
        out = F.relu(out)
        out = self.fc2(out)
        out = F.relu(out)
        out = self.fc3(out)
        return out 

# **Block pixel transformation function**

In [0]:
def apply_block_pixel(entry, p=0, show_img = False):
    if (type(entry) != torch.Tensor):
        entry = torch.tensor(entry).permute(2,0,1) 
    copy = entry.clone()

    if len(entry.shape) == 2:
        copy = copy.unsqueeze(dim = 0)
    a, b, c = copy.size()
    mask = np.zeros(shape = copy.shape)
    
    
    mask = torch.tensor(np.expand_dims(np.random.rand(b,c) < p, 0).astype(float)).byte()
    copy[mask] = 0

    
    #plt.imshow(transforms.ToPILImage()(copy))                
    return copy, mask

In [0]:
def transform(data):
        # Remove the number of channels
        unbind_tuple = torch.unbind(data, 1)
        #convert tuple to tensor
        for var in unbind_tuple:
          unbind_tensor = var
        # Remove the batch size and map it to transformation function
        block_pixel =list(map(apply_block_pixel, torch.unbind(unbind_tensor, 0)))
        # Stack the unbinded tensors
        output = torch.stack(list(zip(*block_pixel))[0],0)
        return output

# **Function for inception score**

In [0]:
from scipy.stats import entropy

def inception_score(imgs, cuda=True, batch_size=32, resize=False, splits=1):
  
    N = len(imgs)
    #print(N)
    #print(type(imgs))

    assert batch_size > 0
    assert N > batch_size

    # Set up dtype
    if cuda:
        dtype = torch.cuda.FloatTensor
    else:
        if torch.cuda.is_available():
            print("WARNING: You have a CUDA device, so you should probably set cuda=True")
        dtype = torch.FloatTensor

    # Set up dataloader
    dataloader = torch.utils.data.DataLoader(imgs, batch_size=batch_size)

    inception_model = BetterCNN()
    inception_model.load_state_dict(torch.load('/content/gdrive/My Drive/Colab Notebooks/bettercnn.weights'))
    inception_model.eval();

   
    def get_pred(x):
        if resize:
            x = up(x)
        x = inception_model(x)
        return F.softmax(x).data.cpu().numpy()

    # Get predictions
    preds = np.zeros((N, 10))

    
    #print(imgs.size())
    preds = get_pred(imgs)
    # Now compute the mean kl-div
    split_scores = []

    for k in range(splits):
        part = preds[k * (N // splits): (k+1) * (N // splits), :]
        py = np.mean(part, axis=0)
        scores = []
        for i in range(part.shape[0]):
            pyx = part[i, :]
            scores.append(entropy(pyx, py))
        split_scores.append(np.exp(np.mean(scores)))

       
    return np.mean(split_scores), np.std(split_scores)

# **Loss Function and Optimizers**

In [0]:
batch_size = 128
lr = 0.0002
train_epoch = 25
INPUT_SIZE = 784
SAMPLE_SIZE = 1000
NUM_LABELS = 10

In [0]:
#Initialize weights
def normal_init(m, mean, std):
    if isinstance(m, nn.ConvTranspose2d) or isinstance(m, nn.Conv2d):
        m.weight.data.normal_(mean, std)
        m.bias.data.zero_()

In [0]:
# Create the Discriminator
model_d = ModelD()
# Create the Generator
model_g = ModelG()

#Create BCELoss function
criterion = nn.BCELoss()

#Initialize the input and noise
input = torch.FloatTensor(128, INPUT_SIZE)
noise = torch.FloatTensor(128, (100))

#Create the batch of latent vectors
fixed_noise = torch.FloatTensor(SAMPLE_SIZE, 100).normal_(0,1)
fixed_labels = torch.zeros(SAMPLE_SIZE, NUM_LABELS)


for i in range(NUM_LABELS):
  for j in range(SAMPLE_SIZE // NUM_LABELS):
    fixed_labels[i*(SAMPLE_SIZE // NUM_LABELS) + j, i] = 1.0
    
label = torch.FloatTensor(128)
one_hot_labels = torch.FloatTensor(128, 10)

In [0]:
#model_d.weight_init(mean=0.0, std=0.05)
model_d.cuda()
#model_g.weight_init(mean=0.0, std=0.05)
model_g.cuda()
input, label = input.cuda(), label.cuda()
noise, fixed_noise = noise.cuda(), fixed_noise.cuda()
fixed_labels = fixed_labels.cuda()

#Initialize SGD optimizers for both model_d and model_g
optim_d = optim.SGD(model_d.parameters(), lr=0.01)
optim_g = optim.SGD(model_g.parameters(), lr=0.01)

fixed_noise = Variable(fixed_noise.cuda())
fixed_labels = Variable(fixed_labels.cuda())

#Establish real and fake lables for training
real_label = 1
fake_label = 0

# **Training Discriminator and Generator**

In [0]:
#List to keep track of the the loss 
d_per_epoch_loss = []
g_per_epoch_loss = []

#For each epochs
for epoch_idx in range(25):
  
  epoch_start_time = time.time()
  
  model_d.train()
  model_g.train()
  # learning rate decay
  if (epoch_idx+1) == 15:
        optim_d.param_groups[0]['lr'] /= 10
        optim_g.param_groups[0]['lr'] /= 10
        print("learning rate change!")

  if (epoch_idx+1) == 20:
        optim_d.param_groups[0]['lr'] /= 10
        optim_g.param_groups[0]['lr'] /= 10
        print("learning rate change!")

  d_loss = []
  g_loss = []
  #For each batch in the data_loader
  for batch_idx, (train_data_x, train_labels_y) in enumerate(full_loader):
    ## Update the model_d network
    # Train with all tranformed data batch
    batch_size = train_data_x.size(0)
    # Obtain the transformed data
    train_data_x = transform(train_data_x)
    train_data_x = train_data_x.view(-1, INPUT_SIZE)
   
    train_data_x = train_data_x.cuda()
    train_labels_y = train_labels_y.cuda()

    input.resize_as_(train_data_x).copy_(train_data_x)
    label.resize_(batch_size).fill_(real_label)
    one_hot_labels.resize_(batch_size, NUM_LABELS).zero_()
    one_hot_labels = one_hot_labels.cuda()
    one_hot_labels.scatter_(1, train_labels_y.view(batch_size,1), 1)
    inputv = Variable(input)
    labelv = Variable(label)
    # Forward pass through model_d
    output = model_d(inputv, Variable(one_hot_labels))
    optim_d.zero_grad()
    # Calculate the loss on transformed data batch
    errD_real = criterion(output, labelv)
    # Backward pass through model_d
    errD_real.backward()
    realD_mean = output.data.cpu().mean()
    
    # Train with all generated batch
    one_hot_labels.zero_()
    rand_y = torch.from_numpy(np.random.randint(0, NUM_LABELS, size=(batch_size,1))).cuda()
    one_hot_labels.scatter_(1, rand_y.view(batch_size,1), 1)
    noise.resize_(batch_size, 100).normal_(0,1)
    label.resize_(batch_size).fill_(fake_label)
    noisev = Variable(noise)
    labelv = Variable(label)
    onehotv = Variable(one_hot_labels)
    # Generate the fake image
    g_out = model_g(noisev, onehotv)
    # Classify the fake image
    output = model_d(g_out, onehotv)
    # Calculate the loss on generated data batch
    errD_fake = criterion(output, labelv)
    fakeD_mean = output.data.cpu().mean()
    # Add the gradients from the real_transformed and fake batches
    errD = errD_real + errD_fake
    # Calculate the gradients
    errD_fake.backward()
    # Update model_d
    optim_d.step()

    ## Update the model_g network
    noise.normal_(0,1)
    one_hot_labels.zero_()
    rand_y = torch.from_numpy(np.random.randint(0, NUM_LABELS, size=(batch_size,1))).cuda()
    one_hot_labels.scatter_(1, rand_y.view(batch_size,1), 1)
    label.resize_(batch_size).fill_(real_label)
    onehotv = Variable(one_hot_labels)
    noisev = Variable(noise)
    labelv = Variable(label)
    # Generate the fake image
    g_out = model_g(noisev, onehotv)
    output = model_d(g_out, onehotv)
    errG = criterion(output, labelv)
    optim_g.zero_grad()
    errG.backward()
    # Update model_g
    optim_g.step()
  
  
    d_loss.append(errD)
    g_loss.append(errG)
    
  epoch_end_time = time.time()  
  
  # Save the generated image
  g_out = model_g(fixed_noise, fixed_labels).data.view(SAMPLE_SIZE, 1, 28,28).cpu()
  save_image(g_out,'{}/{}.png'.format(samples_dir, epoch_idx))
  
 
  # Append the losses
  d_per_epoch_loss.append(torch.mean(torch.stack(d_loss),dim=0)) 
  g_per_epoch_loss.append(torch.mean(torch.stack(g_loss),dim=0))
  
  
  # calculate inception score
  if epoch_idx==24:
    print('Inception score:',inception_score(g_out))
  
  
  print('Epoch {} - D loss = {:.4f}, G loss = {:.4f}, Time taken in sec = {:.4f}'.format(epoch_idx, d_per_epoch_loss[epoch_idx], g_per_epoch_loss[epoch_idx],epoch_end_time-epoch_start_time))
  


# **Plotting the loss versus training iteration**

In [0]:
plt.figure(figsize=(10,5))
plt.title("Generator and Discriminator Loss")
plt.plot(g_per_epoch_loss,label="G")
plt.plot(d_per_epoch_loss,label="D")
plt.xlabel("iterations")
plt.ylabel("Loss")
plt.legend()
plt.show()